<a href="https://colab.research.google.com/github/tahmidjamal12/Vision_Thread/blob/main/Vision_Thread_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### importing libraries and plotting functions

In [ ]:
#import libraries
import torch
from torch import nn

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor


#graphin
import matplotlib.pyplot as plt

# adding a timing function
from timeit import default_timer as timer
def print_train_time(start: float,
                     end:float,
                     device: torch.device = None):
  total_time = end-start
  print(f"Train time on {device}: {total_time:.3f} seconds")
  return total_time

In [ ]:
#plotting functions

def plot_predictions(
    train_data, train_labels, test_data, test_labels, predictions=None
):
    """
  Plots linear training data and test data and compares predictions.
  """
    plt.figure(figsize=(10, 7))

    # Plot training data in blue
    plt.scatter(train_data, train_labels, c="b", s=4, label="Training data")

    # Plot test data in green
    plt.scatter(test_data, test_labels, c="g", s=4, label="Testing data")

    if predictions is not None:
        # Plot the predictions in red (predictions were made on the test data)
        plt.scatter(test_data, predictions, c="r", s=4, label="Predictions")

    # Show the legend
    plt.legend(prop={"size": 14})


def plot_decision_boundary(model: torch.nn.Module, X: torch.Tensor, y: torch.Tensor):
    """Plots decision boundaries of model predicting on X in comparison to y.

    Source - https://madewithml.com/courses/foundations/neural-networks/ (with modifications)
    """
    # Put everything to CPU (works better with NumPy + Matplotlib)
    model.to("cpu")
    X, y = X.to("cpu"), y.to("cpu")

    # Setup prediction boundaries and grid
    x_min, x_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1
    y_min, y_max = X[:, 1].min() - 0.1, X[:, 1].max() + 0.1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 101), np.linspace(y_min, y_max, 101))

    # Make features
    X_to_pred_on = torch.from_numpy(np.column_stack((xx.ravel(), yy.ravel()))).float()

    # Make predictions
    model.eval()
    with torch.inference_mode():
        y_logits = model(X_to_pred_on)

    # Test for multi-class or binary and adjust logits to prediction labels
    if len(torch.unique(y)) > 2:
        y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1)  # mutli-class
    else:
        y_pred = torch.round(torch.sigmoid(y_logits))  # binary

    # Reshape preds and plot
    y_pred = y_pred.reshape(xx.shape).detach().numpy()
    plt.contourf(xx, yy, y_pred, cmap=plt.cm.RdYlBu, alpha=0.7)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.RdYlBu)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())

def accurate_fn(x_seen, x_pred):
  correct = torch.eq(x_seen, x_pred).sum().item()
  return (correct/len(x_pred)) * 100

## Getting DataSet from MNIST database

In [ ]:
train_data = datasets.FashionMNIST(
    root="data", #where to download data to
    train=True, #do we want training set
    download=True, #download yes/no
    transform=ToTensor(), #how do we transform our data
    target_transform=None # how do we want to transform theh labels/target
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
    target_transform=None
)
class_names = train_data.classes

100%|██████████| 26421880/26421880 [00:02<00:00, 10676590.48it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 165937.10it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:06<00:00, 722305.39it/s] 


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 6401505.19it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



### Preparing the DataLoader

In [ ]:
from torch.utils.data import DataLoader

#setup batch size hyperparam
BATCH_SIZE=64

#turn data to batches
train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

test_dataloader = DataLoader(dataset=test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False)

### device agnostic code

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

## Creating CNN Model

In [ ]:
class FashionMNISTModelV0(nn.Module):
  def __init__(self,
               input_shape:int,
               hidden_units:int,
               output_shape:int):
    super().__init__()
    self.conv_block_1= nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1), #hyperparameters
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                 out_channels=hidden_units,
                 kernel_size=3,
                 stride=1,
                 padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*49,
                  out_features=output_shape)
    )

  def forward(self, x):
    return self.classifier(self.conv_block_2(self.conv_block_1(x)))

### Instantiate the model

In [ ]:
torch.manual_seed(69)
Fashion_model = FashionMNISTModelV0(input_shape=1,
                              hidden_units=16,
                              output_shape=len(class_names)).to(device)

### Functionizing training and testing

In [ ]:
# training loop
def train(model: nn.Module,
          data_loader: torch.utils.data.DataLoader,
          loss_fn: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          accuracy_fn,
          device: torch.device = device):
  train_loss, train_acc = 0, 0

  model.train()
  for batch, (X, y) in enumerate(data_loader): #(image, label)

    X, y = X.to(device), y.to(device)
    #forward pass
    y_pred = model(X)

    #calculate da loss
    loss = loss_fn(y_pred, y)
    train_loss += loss
    train_acc += accuracy_fn(y, y_pred.argmax(dim=1)) #logits to pred labels

    #optimizer zero grad
    optimizer.zero_grad()

    #backpropagation
    loss.backward()

    #optimizer step this specifically optimizes per batch even better than once epoch
    optimizer.step()

    if batch % 400 == 0:
      print(f"looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples.")

  #averages train loss
  train_loss /= len(data_loader)
  train_acc /= len(data_loader)

  print(f"Train loss: {train_loss:.5f} | Train acc: {train_acc:.2f}")


In [ ]:
# testing loop
def test(model: nn.Module,
          data_loader: torch.utils.data.DataLoader,
          loss_fn: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          accuracy_fn,
          device: torch.device = device):

  test_loss, test_acc = 0,0
  model.eval()
  with torch.inference_mode():
    for X_test, y_test in data_loader:
      X_test, y_test = X_test.to(device), y_test.to(device)

      #forward pass
      test_pred = model(X_test)

      #calculate da loss
      test_loss += loss_fn(test_pred, y_test)

      #accuracy
      test_acc += accuracy_fn(y_test, test_pred.argmax(dim=1))

    #calculate test and acc loss avg
    test_loss /= len(data_loader)
    test_acc /= len(data_loader)

  print(f"Test_loss: {test_loss:.4f} | Test acc: {test_acc:.4f}")

### Setup optimizer and loss function


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=Fashion_model.parameters(), lr= 0.1)

## Training and testing loop

In [ ]:
# train model

torch.manual_seed(69)
train_time_start_on_device = timer()

epochs = 3

for epoch in range(epochs):
  #train da model
  train(model=Fashion_model,
        data_loader= train_dataloader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy_fn=accurate_fn,
        device=device)

  #test da model
  test(model=Fashion_model,
        data_loader= test_dataloader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy_fn=accurate_fn,
        device=device)

  train_time_end_on_device = timer()
  total_time = print_train_time(start=train_time_start_on_device, end=train_time_end_on_device, device=device)

looked at 0/60000 samples.
looked at 25600/60000 samples.
looked at 51200/60000 samples.
Train loss: 0.65006 | Train acc: 76.62
Test_loss: 0.4999 | Test acc: 81.0609
Train time on cuda: 11.371 seconds
looked at 0/60000 samples.
looked at 25600/60000 samples.
looked at 51200/60000 samples.
Train loss: 0.35091 | Train acc: 87.37
Test_loss: 0.4228 | Test acc: 83.7978
Train time on cuda: 20.670 seconds
looked at 0/60000 samples.
looked at 25600/60000 samples.
looked at 51200/60000 samples.
Train loss: 0.30898 | Train acc: 88.87
Test_loss: 0.3188 | Test acc: 88.7938
Train time on cuda: 30.649 seconds
